# WebScrapping de HowLongToBeat.com

In [1]:
#Librerias
import re
import pandas as pd
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import time

In [2]:
# Funcion get para tratar las excepciones

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    head = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/59.0.3071.115 Safari/537.36'}
    try:
        with closing(get(url, stream=True, headers=head)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

## Requests HTTP

In [ ]:
#Genero una lista con todos los ids que podria llegar a tener la pagina
ids = list(range(60000))

In [47]:
%%time
##Loopeo sobre los id de la URL
games = []


for id in ids:
    try:
        url = 'https://howlongtobeat.com/game.php?id='+str(id)
        raw_html = simple_get(url)
        html = BeautifulSoup(raw_html, 'html.parser')

        ## Titulo del juego
        titulo = html.find('div', {'class':"profile_header shadow_text"}).text.split('\n')
        titulo = titulo[1].rstrip()
            #Pasamos a dict
        titulo_dict = {'Id': id,'Titulo' : titulo}


        ## Rating
        det=html.find('div', {'class':'profile_details'})
        details = []
        for i, li in enumerate(det.select('li')):
            dicc = {i: li.text}
            details.append(li.text)
        rating = details[4][0:2]
            #Pasamos a dict
        rating_dict = {'Rating': rating}
        rating_dict


        ## Completion Times
        tiempos = html.find_all('li',class_=re.compile("short time_"))
            #Descompongo cada uno de los tiempos
        times = []
        for n in range(4):
            x = tiempos[n].select('div')
            x = str(x[0])
            x = x.replace('<div>', '')
            x = x.replace('</div>','')
            x = x.rstrip()
            times.append(x)
            #Creo un diccionario con la clave de cada tiempo
        times_dict = {'Main Story'    : times[0],
                      'Main + Extras' : times[1],
                      'Completionist' : times[2],
                      'All Styles'    : times[3]}


        ## Info del juego
        prof = html.find_all('div', {'class':'profile_info'})
            #Creo un diccionario con la info
        info_dict = {}
        for i in range(len(prof)):
            info_dict[str(prof[i].contents[1]).replace('<strong>','').replace('</strong>','').replace('\n','').replace(':','').rstrip()]\
            = prof[i].contents[2].replace('\n','').rstrip()

        ## Junto Todos los diccionarios en uno solo

        games.append({**titulo_dict, **rating_dict, **times_dict, **info_dict})
        time.sleep(2) # Esperar dos segundos entre cada loop
    except:
        pass

Wall time: 6.3 s


In [48]:
#Chequeo como quedo la lista de diccionarios
games

[{'Id': 50000,
  'Titulo': 'The Elmian Warrior',
  'Rating': 'NR',
  'Main Story': '--',
  'Main + Extras': '--',
  'Completionist': '--',
  'All Styles': '--',
  'Developer': 'The MegaByte',
  'Publisher': 'The MegaByte',
  'Genre': 'Action',
  'NA': ' July 20, 2017',
  'EU': ' July 20, 2017',
  'Updated': ' 3.5 Weeks Ago'},
 {'Id': 27100,
  'Titulo': 'Red Dead Redemption 2',
  'Rating': '94',
  'Main Story': '40 Hours',
  'Main + Extras': '62 Hours',
  'Completionist': '157 Hours',
  'All Styles': '54½ Hours',
  'Developer': 'Rockstar Studios',
  'Publisher': 'Rockstar Games',
  'Playable On': 'PlayStation 4, Xbox One',
  'Genres': 'Third-Person, Action, Adventure, Open World, Shooter',
  'NA': ' October 26, 2018',
  'EU': ' October 26, 2018',
  'Updated': ' 16 Secs Ago'}]

In [56]:
# Convierto la lista en un dataframe
games_df = pd.DataFrame(games)

In [57]:
#Reordeno las columnas para una mejor lectura
games_df = games_df[['Titulo','Main Story','Main + Extras','Completionist','All Styles','Rating','Developer', 'Publishers', 'Genres']]

In [58]:
games_df

,Titulo,Main Story,Main + Extras,Completionist,All Styles,Rating,Developer,Publishers,Genres
0,The Elmian Warrior,--,--,--,--,NR,The MegaByte,The MegaByte,"July 20, 2017"
1,Sphere Frustration,--,--,--,--,NR,GreatherGames,GreatherGames,"July 14, 2017"
2,Shn!p,--,--,--,--,NR,Martin Knopf,Martin Knopf,"July 14, 2017"
3,Chromatic (2017),--,--,--,--,NR,Marc-Antoine Jacob,MAJ Studio,"July 17, 2017"
4,Too Angry to Space,--,--,--,--,NR,Ak84c,Ak84c,"August 08, 2016"
5,ISEE,--,--,--,--,NR,"Half-Face Production, Nova Dimension",Nova Dimension,"July 31, 2017"
6,The House of Da Vinci,5 Hours,--,7 Hours,5½ Hours,80,Blue Brain Games,"Android, iOS, PC",2.5 Weeks Ago
7,Achievement Hunter: Offensive,--,--,3 Hours,3 Hours,40,Putilin Industries,Putilin Industries,"September 15, 2017"
8,Theseus (2017),2 Hours,--,--,2 Hours,60,Forge Reply,Forge Reply,"Action, Adventure"
9,A Mortician's Tale,56 Mins,1 Hours,1 Hours,1 Hours,68,Laundry Bear Games,Laundry Bear Games,Simulation


In [59]:
#Genero un archivo CSV con los resultados del scrapping. Uso como separador pipes, ya que los juegos con mas de un Developer o
# publisher, estan separados por comas

games_df.to_csv('C:/Users/rharari/Desktop/Data Science/DB HLTB.csv', sep='|')

In [ ]:
# Primer loop 6hs 56 min
# Segundo Loop 5hs 35 min
# Tercer loop 4hs 56min
# Cuarto loop 9hs 25 min con 20.000
# Quinto loop 3hs 59 min